In [ ]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [58]:
data = pd.read_excel('excel1.xlsx', sheet_name = 'Data')

In [59]:
%%time

price_df_list = []
for i in tqdm(range(120)):
    price_data = data.iloc[:,3*i:3*i+2].copy()
    price_data.dropna(inplace=True)
    price_data.iloc[:,0] = pd.to_datetime(price_data.iloc[:,0], unit= 'D', origin= '1899-12-30')
    price_data.rename(columns = {price_data.columns[0] : 'Date', price_data.columns[1]: price_data.columns[0]}, inplace = True)
    price_data.set_index('Date', inplace = True)
    price_df_list.append(price_data)

print(price_data)

agg_price_data = price_df_list[0]
for df in tqdm(price_df_list[1:]):
    agg_price_data = agg_price_data.join(df, how='outer', sort = True)

agg_price_data = agg_price_data[(agg_price_data.index >= '31/12/2018') & (agg_price_data.index <= '31/12/2020')].copy()

100%|██████████| 120/120 [00:00<00:00, 300.78it/s]


            NXI FP Equity
Date                     
2010-12-31        26.4957
2011-01-03        26.8603
2011-01-04        27.2288
2011-01-05        27.1047
2011-01-06        27.1861
...                   ...
2021-09-01        43.5800
2021-09-02        43.3600
2021-09-03        43.6600
2021-09-06        43.6200
2021-09-07        43.6800

[2734 rows x 1 columns]


100%|██████████| 119/119 [00:00<00:00, 454.50it/s]

CPU times: user 529 ms, sys: 90.4 ms, total: 620 ms
Wall time: 706 ms



/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:60: UserWarning: Parsing '31/12/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.ge)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:52: UserWarning: Parsing '31/12/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.le)


In [ ]:
agg_price_data

In [ ]:
for ticker in tqdm(agg_price_data.columns):
    series = agg_price_data[ticker]
    first = series.first_valid_index()
    last = series.last_valid_index()
    series_real = series[(series.index >= first) & (series.index <= last)].fillna(method = 'bfill')
    agg_price_data[ticker][(agg_price_data[ticker].index >= first) & (agg_price_data[ticker].index <= last)] = series_real

In [ ]:
for ticker in tqdm(agg_price_data.columns):
    if agg_price_data[ticker].isnull().values.any():
        print(ticker)
        nulls = agg_price_data[ticker].isnull()
        print(nulls[nulls == True].index)

- GTO -> Gemalto bought by Thales
- ING -> Ingenico bought by Worldline
- ALT -> Altran became subsidiary of Capgemini

In [ ]:
plt.plot(agg_price_data['GTO NA Equity'])
plt.show()

In [ ]:
plt.plot(agg_price_data['ING FP Equity'])
plt.show()

In [ ]:
plt.plot(agg_price_data['ALT FP Equity'])
plt.show()

In [ ]:
(agg_price_data - agg_price_data.shift(1))/agg_price_data.shift(1)

In [ ]:
agg_price_data.pct_change()

In [ ]:
returns = agg_price_data.pct_change().dropna() # set returns to 0 when the stock disappears
returns

In [ ]:
df_mkt_cap_list = []
for i in tqdm(range(120)):
    mkt_cap_data = data.iloc[:,[3*i,3*i+2]].copy()
    mkt_cap_data.dropna(inplace = True)
    mkt_cap_data.iloc[:,0] = pd.to_datetime(mkt_cap_data.iloc[:,0], unit= 'D', origin= '1899-12-30')
    mkt_cap_data.rename(columns = {mkt_cap_data.columns[0] : 'Date', mkt_cap_data.columns[1]: mkt_cap_data.columns[0]}, inplace = True)
    mkt_cap_data.set_index('Date', inplace = True)
    df_mkt_cap_list.append(mkt_cap_data)

agg_mkt_cap_data = df_mkt_cap_list[0]
for df in tqdm(df_mkt_cap_list[1:]):
    agg_mkt_cap_data = agg_mkt_cap_data.join(df, how='outer', sort = True)

In [ ]:
agg_mkt_cap_data

In [ ]:
agg_mkt_cap_data = agg_mkt_cap_data[(agg_mkt_cap_data.index >= '31/12/2018') & (agg_mkt_cap_data.index <= '31/12/2020')].copy()

In [ ]:
for ticker in tqdm(agg_mkt_cap_data.columns):
    if agg_mkt_cap_data[ticker].isnull().values.any():
        print(ticker)
        nulls = agg_mkt_cap_data[ticker].isnull()
        print(nulls[nulls == True].index)

In [ ]:
agg_mkt_cap_data['ALO FP Equity']['11-2020']

In [ ]:
agg_mkt_cap_data['ALO FP Equity']['11-2020']/agg_price_data['ALO FP Equity']['11-2020']

In [ ]:
agg_mkt_cap_data['ALO FP Equity'].fillna(method = 'bfill', inplace = True)
agg_mkt_cap_data['ALO FP Equity']['11-2020']

In [ ]:
agg_mkt_cap_data.fillna(value = 0.0, inplace = True)

In [ ]:
agg_mkt_cap_data.isnull().values.any()

In [ ]:
from strategy import Strategy

In [ ]:
sg_returns = agg_price_data['GLE FP Equity'].loc['2019':].pct_change().dropna()
longonly_sg = Strategy('Long Only SG', sg_returns)

In [ ]:
type(longonly_sg)
longonly_sg.sharpe_ratio()

In [ ]:
longonly_sg.illustrate()

In [ ]:

from strategy import CapiWeighted
cw_port = CapiWeighted('CW_all', returns, agg_mkt_cap_data)
cw_port.illustrate()